# Form the NB Regression Random Intercept Model: Simulated set

Next we may try joint estimation using Newton's + IPOPT. 

    I create new structures `NBCopulaVCObs`, `NBCopulaVCModel`
    



In [81]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.5

r = 100
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Vector{Vector{Float64}}:
 [35.0, 32.0, 49.0, 37.0, 53.0]
 [55.0, 40.0, 39.0, 45.0, 35.0]
 [34.0, 42.0, 43.0, 60.0, 34.0]
 [41.0, 39.0, 34.0, 43.0, 55.0]
 [25.0, 60.0, 26.0, 32.0, 53.0]
 [42.0, 41.0, 46.0, 44.0, 47.0]
 [58.0, 47.0, 45.0, 42.0, 43.0]
 [48.0, 52.0, 39.0, 37.0, 56.0]
 [43.0, 33.0, 38.0, 39.0, 38.0]
 [44.0, 44.0, 39.0, 37.0, 36.0]
 [34.0, 36.0, 45.0, 35.0, 39.0]
 [39.0, 50.0, 51.0, 32.0, 42.0]
 [34.0, 47.0, 36.0, 50.0, 37.0]
 ⋮
 [49.0, 46.0, 53.0, 42.0, 34.0]
 [54.0, 46.0, 48.0, 36.0, 49.0]
 [49.0, 54.0, 38.0, 45.0, 32.0]
 [36.0, 47.0, 53.0, 54.0, 45.0]
 [34.0, 39.0, 42.0, 41.0, 59.0]
 [35.0, 38.0, 37.0, 49.0, 48.0]
 [35.0, 46.0, 53.0, 29.0, 34.0]
 [30.0, 56.0, 48.0, 46.0, 40.0]
 [40.0, 47.0, 35.0, 38.0, 48.0]
 [45.0, 37.0, 37.0, 28.0, 40.0]
 [29.0, 35.0, 53.0, 44.0, 46.0]
 [47.0, 48.0, 54.0, 38.0, 47.0]

In [82]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);
# gcm.r[1] = r

In [83]:
initialize_model!(gcm)
@show gcm.β
# @show gcm.Σ
@show gcm.r

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -183965.880323221 39999
2 -183965.880323221 -183965.880323221 39999
initializing variance components using MM-Algorithm
gcm.Σ = [225.63027521074963, 1857.7441889422705]
hiiii gcmPoisson.Σ = [225.63027521074963, 1857.7441889422705]
gcm.θ = [-5.404042280046205, 224.7201241053222, 1864.4405333056088]
gcm.θ = [3.76335082451623, 225.63027521074963, 1857.7441889422705]
gcm.θ = [3.76335082451623, 225.63027521074963, 1857.7441889422705]
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Starting derivative checker for first derivatives.

* grad_f[          2] = -1.1624128938060323e+01    ~ -1.1610857440697405e+01  [ 1.143e-03]
* grad_f[          3] =  1.4000239622658115e+00    ~  1.3984520467760686e+00  [ 1.124e-03]

Derivative checker detected 2 error(s).

Number of nonzeros i

1-element Vector{Float64}:
 115.79177126877832

In [84]:
β_true

3.757872325600888

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [85]:
GLMCopula.loglikelihood!(gcm, true, true)

-178887.0205092799

## Try Quasi-Newton after specifying gradient of r

to-do: 

1. added field $r$ to model structure $gcm.r$, change access to r this way instead of through the distribution
2. add $gc.\nabla r$ in loglikelihood function


## Newton after specifying the hessian term too

3. add $gc.Hr$ in loglikelihood function

In [86]:
# Quasi-Newton
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100,
#     tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo",
#     mehrotra_algorithm="yes", warm_start_init_point="yes",
    hessian_approximation = "limited-memory"))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_nb.jl:31


-178546.85390244896

In [87]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.757598951146338; true β = 3.757872325600888
estimated variance component 1 = 0.13767175305165796; true variance component 1 = 0.1
estimated variance component 2 = 0.7299223586404632; true variance component 2 = 0.5
estimated r = 96.67669395953683; true r = 100


In [69]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.454302942507524; true β = 1.4552872326068422
estimated variance component 1 = 0.16080189197837352; true variance component 1 = 0.1
estimated variance component 2 = 0.8976070963094536; true variance component 2 = 0.5
estimated r = 11.367324765405236; true r = 10


In [62]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.0866771907017754; true β = 1.09861228866811
estimated variance component 1 = 0.16508355574885977; true variance component 1 = 0.1
estimated variance component 2 = 1.0080802223132954; true variance component 2 = 0.5
estimated r = 8.138290481765807; true r = 7


**If initializing VCs to 1**:
+ With true $r = 1$
    - good estimates of $\beta$, VC parameters, and r
    - sometimes does crazy line search. 
+ With true $r = 2$
    - VC parameters goes to 0, esimated $\beta$ is wrong, and estimated r is 1.17. 

**If initializing VCs by update_$\Sigma$**:
+ With true $r = 1$
    - one VC parameters becomes basically 0 after initialization
    - bad estimated beta (-0.45 vs -0.84 truth)
    - estimated VC = 0.0 and 0.69 (truth = 0.1 and 0.5)
    - estimated r = 1.1

## Newton

In [8]:
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, 
    tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo",
    hessian_approximation = "exact"))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        4
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_nb.jl:30


-118498.19010251868

In [9]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.431700604659203; true β = 1.4552872326068422
estimated variance component 1 = 0.14165778449786529; true variance component 1 = 0.1
estimated variance component 2 = 0.5807584133259613; true variance component 2 = 0.5
estimated r = 5.676937046097018; true r = 10


In [10]:
gcm.∇θ

4-element Vector{Float64}:
 2121.5670801354186
  -71.10375733934279
   12.900327891814868
  443.90634061495734